In [1]:
import pandas as pd
from Classifier.BTree import BTree
from Classifier.Visualize import visualize_tree, visualize_modeltree
from Classifier.BTreeTraversal import BTreeTraversal
from Classifier.adtReSplit import ReSplit, Choose_leaf, CrossSplit
import pickle
import argparse
import os
import time
import matplotlib.pyplot as plt
import warnings
import scanpy as sc
warnings.filterwarnings("ignore")
import numpy as np

In [2]:
def dfs(node, adata, merge_cutoff, prior_gene):
    if node.key == ('leaf',):
        adata_sub = adata[list(set(node.indices)&set(adata.obs_names)),:]
        print(adata_sub.shape)
        # print(len(node.indices),adata_sub.X.shape)
        # sc.pp.scale(adata_sub, max_value=10)
        # sc.tl.pca(adata_sub, n_comps=10)
        # node.stop = None
        node, bic_list, min_bic_node = Choose_leaf(data=adata_sub,prior_gene=prior_gene,merge_cutoff=merge_cutoff,use_parent=True) 
        return node       
    else:
        node.left = dfs(node.left, adata, merge_cutoff, prior_gene)
        node.right = dfs(node.right, adata, merge_cutoff, prior_gene)
        return node

In [3]:
adt_path = [
            '../SeuratV4/subdata/4_10_ADT.csv',
            '../SeuratV4/subdata/4_11_ADT.csv','../SeuratV4/subdata/4_12_ADT.csv']#'../SeuratV3/subdata/5_4_ADT.csv',
            
rna_path = [
            '../SeuratV4/subdata/4_10_RNA.h5ad',
            '../SeuratV4/subdata/4_11_RNA.h5ad', '../SeuratV4/subdata/4_12_RNA.h5ad']#'../SeuratV3/subdata/5_4_RNA.h5ad',
            

output_path = '../output/7_1'
merge_cutoff = 0.15
compact_flag = True
current_tree = None #['7_1_0','7_1_1','7_1_2']


    
starttime = time.time()
print('read data and run CITE-sort.')

adtdata = {}
rnadata = {}
for i in range(len(adt_path)):
    if adt_path[i] != None:
        adtdata[i] = pd.read_csv(adt_path[i], header = 0, index_col=0, sep=',')
    rnadata[i] = sc.read_h5ad(rna_path[i])
    rnadata[i].var_names_make_unique()

# prior_gene = dict({'exhausted':['TOX','TOX2','LAG3','BTLA','CBLB','ITCH','NEDD4'],
# 'TCM/TEM':['CCR7','CD44','IL7R','IL15RA','MBD2'],'Th17':['KLRB1','IL23R','CCR6','IL1R1','STAT3']})
prior_gene = {}
if current_tree != None:
    f = open(current_tree+'tree.pickle','rb')
    tree = pickle.load(f)
    f.close()
    data_sub = adtdata.loc[list(set(tree.indices)&set(adtdata.index)),:]
    tree = dfs(tree, data_sub, merge_cutoff,rnadata)
else:
    # nodelist=[BTree(('leaf',)) for i in range(len(adt_path))]
    crossnode = CrossSplit(adtdata.copy(),merge_cutoff, weight=np.ones(len(adt_path)),rnadata=rnadata.copy())
        
    # tree, bic_list, min_bic_node = Choose_leaf(data=data,merge_cutoff=merge_cutoff,rawdata=data.copy(),datarna=data_rna)

def inner_dfs(node, crossnode, i):
    if node != None and crossnode.left != None:
        # print(crossnode.left.nodelist,crossnode.right.nodelist, node.key)
        node.left = inner_dfs(crossnode.left.nodelist[i], crossnode.left, i) 
        node.right = inner_dfs(crossnode.right.nodelist[i], crossnode.right, i)
    return node

def modeltree_dfs(node, crossnode):
    if node != None and crossnode.left != None:
        node.left = modeltree_dfs(crossnode.left.modelnode,  crossnode.left)
        node.right = modeltree_dfs(crossnode.right.modelnode,  crossnode.right)
    return node

for i in range(len(adtdata)):
    tree = inner_dfs(crossnode.nodelist[i], crossnode, i)
    output = output_path+'_'+str(i+1)
    if not os.path.exists(output):
        os.mkdir(output)
    visualize_tree (tree, adtdata[i], output, 'tree', compact=compact_flag, rnadata=rnadata[i].copy())

    f = open(output+'/tree.pickle','wb')
    pickle.dump(tree,f)
    f.close()

    # print('generate labels.')

    traversal = BTreeTraversal(tree,save_min_BIC=False)
    leaves_labels = traversal.get_leaf_label()
    leaves_labels.to_csv(output + '/leaf_labels.csv')

tree = modeltree_dfs(crossnode.modelnode, crossnode)
visualize_modeltree(tree, output_path+'_0', 'tree')
f = open(output_path+'_0'+'/tree.pickle','wb')
pickle.dump(tree,f)
f.close()

endtime = time.time()

print('Time using: ', round(endtime-starttime, 3),'secs')
# plt.plot(list(range(len(bic_list))), bic_list)
# plt.savefig('BIC_as_split.png')



read data and run CITE-sort.
best: ('CD14',) in {('CD14',): 17.40757690657118, ('CD8',): 7.915855777321983, ('CD4-1',): -2.3558129073224148, ('CLEC12A',): -1.4785970400245132, ('CD27',): 11.742002659630298, ('CD19',): -4.497470238077325}
best: ('CD3-1',) in {('CD8',): -7.808692060554937, ('CD4-1',): -11.224140543133306, ('CD27',): 5.689020343615088, ('CD3-1',): 5.777486349421036, ('CD19',): -4.291760590418036}
best: ('CLEC12A',) in {('CD8',): 11.809917828334209, ('CD4-1',): 1.441022036136248, ('CLEC12A',): 17.60710185963338}
two dimensional rescan features 2
best: ('CD3-1', 'CD27') in {('CD3-1', 'CD27'): 5.935813362508053, ('CD3-1',): 5.920964193346752, ('CD4-1',): -4.034903739056244, ('CD19',): -995.0}


IndexingError: Too many indexers

In [15]:
a = ('a','b',)
('x',)+a
# a.append(None)
# a.append(None)

('x', 'a', 'b')

In [14]:
import scanpy as sc
import numpy as np

adata = sc.AnnData(X=np.array([[1,2,3],[4,5,6]]))
adata.raw = adata
sc.pp.log1p(adata)
adata = adata.raw

/tmp/ipykernel_3426567/3464284222.py:4: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = sc.AnnData(X=np.array([[1,2,3],[4,5,6]]))


In [26]:
from Classifier.gcca import GCCA
import logging
import numpy as np

# set log level
logging.root.setLevel(level=logging.INFO)

# create data in advance
a = np.random.rand(50, 50)
b = np.random.rand(50, 60)
c = np.random.rand(50, 70)
d = np.random.rand(50, 80)
e = np.random.rand(50, 90)
f = np.random.rand(50, 100)
g = np.random.rand(50, 110)
h = np.random.rand(50, 120)
i = np.random.rand(50, 130)
j = np.random.rand(50, 140)
k = np.random.rand(50, 150)

# create instance of GCCA
gcca = GCCA(n_components=10)
# calculate GCCA
gcca.fit(a, b, c, d, e, f, g, h, i, j, k)
# transform
U = gcca.transform(a, b, c, d, e, f, g, h, i, j, k)

h = gcca.h_list
# save
# gcca.save_params("gcca.h5")
# # load
# gcca.load_params("gcca.h5")
# # plot
# gcca.plot_gcca_result()


2023-10-06 19:36:57,509 : Classifier.gcca : INFO : data num is 11
2023-10-06 19:36:57,509 : Classifier.gcca : INFO : data shape x_0: (50, 50)
2023-10-06 19:36:57,510 : Classifier.gcca : INFO : data shape x_1: (50, 60)
2023-10-06 19:36:57,511 : Classifier.gcca : INFO : data shape x_2: (50, 70)
2023-10-06 19:36:57,512 : Classifier.gcca : INFO : data shape x_3: (50, 80)
2023-10-06 19:36:57,513 : Classifier.gcca : INFO : data shape x_4: (50, 90)
2023-10-06 19:36:57,514 : Classifier.gcca : INFO : data shape x_5: (50, 100)
2023-10-06 19:36:57,515 : Classifier.gcca : INFO : data shape x_6: (50, 110)
2023-10-06 19:36:57,515 : Classifier.gcca : INFO : data shape x_7: (50, 120)
2023-10-06 19:36:57,516 : Classifier.gcca : INFO : data shape x_8: (50, 130)
2023-10-06 19:36:57,517 : Classifier.gcca : INFO : data shape x_9: (50, 140)
2023-10-06 19:36:57,518 : Classifier.gcca : INFO : data shape x_10: (50, 150)
2023-10-06 19:36:57,519 : Classifier.gcca : INFO : normalizing
2023-10-06 19:36:57,520 : Cl

In [35]:
import matplotlib.pyplot as plt
# i=0
for i in range(len(U)-10):
    u = h[i]
    # plt.scatter(u[:,0],u[:,1],s=2)
    print(u.shape)
    print(u[0])
    # plt.hist(u[0],bins=100)
    # plt.legend()
    # i= i +1

(50, 539)
[-0.24728392  0.23471455  0.67194189  0.07103813  0.11997297 -0.38995011
 -0.42884888 -0.35609596 -0.22158014  0.16224456 -0.9689693  -0.46274974
  0.48912635 -0.25093527  0.18239752  0.4120983  -0.28479214 -0.41038887
  0.21740309  0.86199671 -0.25461973  0.70420907  0.79621562  0.16575202
  0.32335856 -1.79721834 -0.43442068  0.04130927  0.73050664 -0.01105656
  0.00424797  0.80727731  0.78162108  0.88969245  1.06768959  0.24057316
 -1.24961011 -1.15878374  0.31419237  0.50792303 -0.30025052 -1.57134849
 -0.65674923  0.65527352  0.05842176 -1.24887359 -1.1320752  -0.56872746
 -0.13313325 -2.37156765 -2.37156765 -2.37156765 -2.37156765  2.37156765
 -2.37156765  2.37156765  2.37156765 -2.37156765  2.37156765 -2.37156765
 -2.37156765  2.37156765 -2.37156765  2.37156765  2.37156765  2.37156765
 -2.37156765 -2.37156765 -2.37156765 -2.37156765  2.37156765  2.37156765
 -2.37156765 -2.37156765 -2.37156765  2.37156765 -2.37156765 -2.37156765
 -2.37156765  2.37156765 -2.37156765 -2.3

In [42]:
p={}
0 in p.keys()

False